In [ ]:
import os
import re
import json
from copy import deepcopy

import fitz
import Levenshtein
import pandas as pd
from openai import OpenAI, ChatCompletion
from pydantic import BaseModel

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/openai_api_mattiavanzetto.txt", encoding='utf-8') as file:
    api_key = file.read()

### PDF to text

Saving all menu pdf content in a dictionary with key name of the restaurant, value text of the menu.

In [ ]:
folder_path = "/Users/vanzettom/Documents/mattia/personale/hackapizza/Hackapizza Dataset/Menu"

text_dict = {}

for i, filename in enumerate(os.listdir(folder_path), start=1):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)

        doc = fitz.open(file_path)
        text = "\n".join([page.get_text("text") for page in doc])
        doc.close()

        file_key = os.path.splitext(filename)[0]
        print(f"{i} - {file_key}")
        text_dict[file_key] = text

Converting symbols related to galaxy orders to text indicating explicitly that the plate can be eaten by certain orders.

In [ ]:
print(text_dict['Datapizza'][2501:2650])

In [ ]:
text_dict = {k: v.replace(" 🪐", "\nQuesto piatto può essere mangiato dai membri dell'Ordine della Galassia di Andromeda.") for k, v in text_dict.items()}
text_dict = {k: v.replace(" 🌱", "\nQuesto piatto può essere mangiato dai membri dell'Ordine dei Naturalisti.") for k, v in text_dict.items()}
text_dict = {k: v.replace(" 🌈", "\nQuesto piatto può essere mangiato dai membri dell'Ordine degli Armonisti.") for k, v in text_dict.items()}

In [ ]:
print(text_dict['Datapizza'][2501:2650])

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/menu_text_dict.json") as f:
    menu_text_dict = json.load(f)

### Menu chunking with regex

I exploit the dish_mapping.json file to create single chunk containing all the text related to a specific dish. To do so I use Levenshtein distance since there are typos.

Firstly I want to create a list of plate for every restaurant

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/Hackapizza Dataset/Misc/dish_mapping.json") as f:
    dish_mapping = json.load(f)    

In [ ]:
dish_mapping

In [ ]:
def contains_fuzzy_by_line(text, query, max_distance=2, lower=True):
    """
    Checks if 'query' is present in the text line by line,
    allowing a maximum Levenshtein distance of 'max_distance'.
    """
    
    if lower:
        query = query.lower()
        text = text.lower()

    lines = text.split("\n")
    
    for line in lines:
        if Levenshtein.distance(line.strip(), query) <= max_distance:
            return True, line
    
    return False, 'no match'

In [ ]:
# example

contains_fuzzy_by_line(text_dict['L Essenza di Asgard'], 'Galassia di Sapore', max_distance=0)

In [ ]:
# list of all dishes
all_dishes = list(dish_mapping.keys())

# dictionary with key restaurant and value an empty list that will contain the list of dishes of the restaurant
dish_list_dict = {k: list() for k in text_dict.keys()}

# list containing all the dishes that need to be searched in the menu texts
to_be_found = deepcopy(all_dishes)

# auxiliary dictionary that will count how many times a dish is found in a menu
# (there are dishes with similar names and wrong dishes could be asisgned to a restaurant)
dish_found_count = {k: 0 for k in dish_mapping.keys()}

# auxiliary dictionary that will containg a key for every dish found in multiple restaurants
ambiguity_dict = dict()

# dictionary containing a key for every restaurant, and as value a dictionary containing a key for every plate wrote with the original name,
# value with name of the plate found in the menu (possibly mispelled) 
mapping_name_found = {k: dict() for k in text_dict.keys()} 


for dish in all_dishes:
    found = False
    max_distance = 0
    while found == False:
        for restaurant, text in text_dict.items():
            match, line = contains_fuzzy_by_line(text, dish, max_distance=max_distance)
            if match:
                dish_found_count[dish] += 1
                print(f"{dish}: found")
                dish_list_dict[restaurant].append(dish)
                mapping_name_found[restaurant][dish] = line
        if dish_found_count[dish] == 0:
            max_distance += 1
        elif dish_found_count[dish] == 1:
            found = True
        elif dish_found_count[dish] > 1:
            found = True
            ambiguity_dict[dish] = [k for k, v in dish_list_dict.items() if dish in v]

In [ ]:
ambiguity_dict

In [ ]:
{k: v for k, v in dish_found_count.items() if v == 0}

In [ ]:
{k: v for k, v in dish_found_count.items() if v > 1}

In [ ]:
mapping_name_found

In [ ]:
for k, v in dish_list_dict.items():
    print(f"\n{k}: ")
    for dish in v:
        print(f"   - {dish}")

It seems that every plate is correctly assigned to its restaurant.

Now I create the chunks:

In [ ]:
def get_chunks(text, dish_list):
    lines = [l.strip() for l in text.lower().split("\n")]
    
    chunks = dict()
    building_first_chunk = True
    current_chunk = list()
    remaining_dishes = list(dish_list)
    key = 'intro'
    
    for line in lines:
        current_chunk.append(line)
        for dish in dish_list:
            if dish == line:
                chunks[key] = "\n".join(current_chunk[:-1])
                #print(chunks[key])
                key = dish
                remaining_dishes.remove(dish)
                current_chunk = list()
                
                
    if current_chunk:
        chunks[key] = "\n".join(current_chunk)
    
    print(f"{len(remaining_dishes)} dishes remain not found: {remaining_dishes}")
    
    return chunks

In [ ]:
all_pdf_chunks = {restaurant: dict() for restaurant in text_dict.keys()}

for i, (restaurant, text) in enumerate(text_dict.items(), start=1):
    print(f"\n{i} - {restaurant}:")
    all_pdf_chunks[restaurant] = get_chunks(text, mapping_name_found[restaurant].values())

In [ ]:
to_remove = """legenda ordini professionali gastronomici
🪐 ordine della galassia di andromeda
🌱 ordine dei naturalisti
🌈 ordine degli armonisti"""

In [ ]:
all_pdf_chunks['Datapizza']['pizza baby simone e alessandro'] = all_pdf_chunks['Datapizza']['pizza baby simone e alessandro'].replace(to_remove, "")
all_pdf_chunks['L Essenza delle Dune']['sfere del ricordo astrale'] = all_pdf_chunks['L Essenza delle Dune']['sfere del ricordo astrale'].replace(to_remove, "")

In [ ]:
for k, v in all_pdf_chunks['L Architetto dell Universo'].items():
    print("key: ", k)
    print(v)
    print("------\n")

In [ ]:
all_pdf_chunks['L Essenza di Asgard'].keys()

In [ ]:
'quadrifonia cosmica: sinfonia di sapori e dimensioni'

In [ ]:
mapping_name_found['L Essenza di Asgard']['Quadrifonia Cosmica: Sinfonia di Sapori e Dimensioni']

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/all_pdf_chunks.json") as f:
    all_pdf_chunks = json.load(f)

### Getting structured info from plate chunks

#### Get chef data

In [ ]:
all_pdf_chunks

In [ ]:
license_list = """
### Licenza Psionica (P):
-Livello 0: Posseduta da tutti se non diversamente specificato. Tipica degli esseri senzienti.
-Livello I: lettura pensiero, telecinesi e teletrasporto di oggetti di massa inferiore a 5 kg, precognizione e visione del passato fino a 5 minuti.
-Livello II: manipolazione della probabilità, telecinesi e teletrasporto di oggetti di massa inferiore a 20 kg, manipolazione delle forze fondamentali dell’universo.
-Livello III: capacità di donare la coscienza e l’intelletto ad oggetti, manipolazione della realtà circoscritta a stanze, teletrasporto senza errore in qualsiasi dimensione temporale, comunione con entità di altri piani.
-Livello IV: proiezione astrale, riscrittura di realtà circoscritta a piccole nazioni o asteroidi.
-Livello V: riscrittura di realtà di intere linee temporali o galassie. Questo livello è equivalente al Grado di influenza di livello tecnologico III (LTK III).

### Licenza Temporale (t):
-Livello I: effetti temporali relativi al presente come dilatazione o accelerazione del tempo.
-Livello II: Livello I + effetti temporali che riguardano linee temporali future.
-Livello III: Livello II + effetti temporali che riguardano linee temporali passate.

### Licenza Gravitazionale (G):
-Livello 0: 5 < G ≤ 10, posseduta da tutti se non diversamente specificato.
-Livello I: 0 < G ≤ 100.
-Livello II: 0 < G ≤ 10^6.
-Livello III: G > 10^6.

### Licenza Antimateria (e+):
-Livello 0: particelle, posseduta da tutti se non diversamente specificato.
-Livello I: antiparticelle.

### Licenza Magnetica (Mx):
-Livello 0: Polo nord e sud, posseduta da tutti se non diversamente specificato.
-Livello I: Mono-polo.

### Licenza Quantistica (Q):
-Livello n: Numero di stati in superposizione dove n è il numero di stati.

### Licenza Luce (c):
-Livello I: Solo colori primari (RGB).
-Livello II: Tutto lo spettro visibile umano.
-Livello III: Tutte le frequenze.

### Livello di Sviluppo Tecnologico (LTK):
-Livello I: Planetario.
-Livello II: Sistema Stellare.
-Livello III: Galassia.
-Livello IV: Superamasso di Galassie.
-Livello V: Intero Universo.
-Livello VI: Universi multipli.
-Livello VI+: Tutte le fonti energia.

"""

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/license_list.json", "w", encoding="utf-8") as f:
    json.dump(license_list, f, ensure_ascii=False, indent=4)

In [ ]:
client = OpenAI(
  api_key=api_key
)

In [ ]:
class ChefData(BaseModel):
    chef_name: str
    license_list: list[str]

In [ ]:
chef_data_dict = {restaurant: dict() for restaurant in all_pdf_chunks.keys()}

for i, (restaurant, menu_chunks_dict) in enumerate(all_pdf_chunks.items(), start=1):
    print(f"{i} - {restaurant}")
    
    output = client.beta.chat.completions.parse(
        model="gpt-4o",
        store=True,
        messages=[
            {
                "role": "system",
                "content": """
Sei un assistente che sta aiutando ad estrarre informazioni da alcuni menù di ristoranti scritti in italiano.
I ristoranti descritti ed i piatti contenuti nei menù sono inventati e si trovano in una ambientazione fantascientifica nello spazio, in diversi sistemi solari.
"""
            },
            {
                "role": "user",
                "content": f"""
Il seguente frammento di testo, preso dal menù del ristorante, descrive un ristorante ed il suo chef.
All'interno del testo sono riportati il nome dello chef e le licenze che possiede.
Restituiscimi una classe python contenente un attributo riportante la stringa con il nome dello chef,
ed un'attributo riportante la lista di stringhe relative alle licenze che possiede lo chef.

Considera che le licenze che è possibile avere sono le seguenti:

{license_list}

Inoltre considera che se nel testo una licenza è riferita al ristorante la devi associare anch'essa allo chef.

### FRAMMENTO DI MENU:
{menu_chunks_dict['intro']}
"""
            }
        ],
        response_format=ChefData,
    )
    

    chef_data_dict[restaurant]['chef_name'] = output.choices[0].message.parsed.chef_name
    chef_data_dict[restaurant]['license_list'] = output.choices[0].message.parsed.license_list

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/chef_data_dict.json") as f:
    chef_data_dict = json.load(f)  

In [ ]:
chef_data_dict

In [ ]:
chef_data_dict['Le Dimensioni del Gusto']

#### license cleaning

In [ ]:
chef_data_dict_clean = dict()

for i, (restaurant, chef_data) in enumerate(chef_data_dict.items(), start=1):

    print(f"{i} - {restaurant}")
    
    output = client.chat.completions.create(
        model="gpt-4o",
        store=True,
        messages=[
            {
                "role": "system",
                "content": """
Sei un assistente che sta aiutando ad pulire e sistemare informazioni da dati relativi chef scritti in italiano.
Il pezzo di testo che vedrai conterrà la lista di licenze possedute da uno specifico chef con il rispettivo livello.
Le licenze potrebbero essere espresse sia con un'abbreviazione sia per intero, sia ad esempio per intero con l'abbreviazione tra parentesi"""
            },
            {
                "role": "user",
                "content": f"""
Il seguente frammento di testo contiene la lista di licenze che uno chef possiede, con il rispettivo livello.
Voglio che mi restituisci solo un dizionario python contenente tante chievi quante le licenze che lo chef possiede.
Ogni chiave deve essere valorizzata con il nome della licenza espresso per intero e tra parentesi l'abbreviazione (es. "Antimateria (e+)", "Livello di Sviluppo Tecnologico (LTK)" ecc).
Ogni valore deve essere valorizzato con un intero riportante il livello della licenza posseduta. 
Ad esempio se uno chef possiede le licenze "licenza LTK III, e+ II", la lista restituità sarà:
[{{"Livello di Sviluppo Tecnologico (LTK)": 3}}, {{"Antimateria (e+)": 2}}]


Considera che nella lista di licenze che ti mostrerò potrebbero esserci dei typo.
Te devi correggerli mentre pulisci il dato e mi restituisci la lista che desidero.

Di seguito trovi tutte le possibili licenze che uno chef può avere
(se capisci che una licenza va assegnata di default ad uno chef ma non è riportata tra quelle che possiede, aggiungila tu): 

## LISTA DI TUTTE LE POSSIBILI LICENZE:
{license_list}

Inoltre considera che se nel testo una licenza è riferita al ristorante la devi associare anch'essa allo chef.

### LICENZE POSSEDUTE DALLO CHEF:

{chef_data_dict[restaurant]['license_list']}
---

Il tuo output deve contenere solamente la lista, senza nessun altro commento o virgolette, in modo che sia possibile fare eval direttamente sul tuo risultato.
"""
            }
        ],
    )
    
    chef_data_dict_clean[restaurant] = dict()
    chef_data_dict_clean[restaurant]['chef_name'] = chef_data_dict[restaurant]['chef_name']
    chef_data_dict_clean[restaurant]['license_list'] = output.choices[0].message.content

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/chef_data_dict_clean.json") as f:
    chef_data_dict_clean = json.load(f)  

In [ ]:
chef_data_dict_clean

### Get restaurant location

In [ ]:
distances = pd.read_csv('/Users/vanzettom/Documents/mattia/personale/hackapizza/Hackapizza Dataset/Misc/Distanze.csv')
distances.shape

In [ ]:
planet_list = ['Tatooine', 'Asgard', 'Namecc', 'Arrakis', 'Krypton', 'Pandora', 'Cybertron', 'Ego', 'Montressosr', 'Klyntar']

In [ ]:
planets = """
- Tatooine
- Asgard
- Namecc
- Arrakis
- Krypton
- Pandora
- Cybertron
- Ego
- Montressosr
- Klyntar
"""

In [ ]:
class RestaurantLocation(BaseModel):
    planet: str

In [ ]:
location_dict = {restaurant: "" for restaurant in all_pdf_chunks.keys()}

for i, (restaurant, menu_chunks_dict) in enumerate(all_pdf_chunks.items(), start=1):
    print(f"{i} - {restaurant}")
    
    output = client.beta.chat.completions.parse(
        model="gpt-4o",
        store=True,
        messages=[
            {
                "role": "system",
                "content": """
Sei un assistente che sta aiutando ad estrarre informazioni da alcuni menù di ristoranti scritti in italiano.
I ristoranti descritti ed i piatti contenuti nei menù sono inventati e si trovano in una ambientazione fantascientifica nello spazio, in diversi sistemi solari.
"""
            },
            {
                "role": "user",
                "content": f"""
Il seguente frammento di testo, preso dal menù del ristorante, descrive un ristorante ed il suo chef.
All'interno del testo è riportato il pianeta su cui è localizzato il ristorante.
Restituiscimi il pianeta in cui è presente il ristorante

Considera che i pianeti possibili sono i seguenti:

{planets}

### FRAMMENTO DI MENU:
{menu_chunks_dict['intro']}
"""
            }
        ],
        response_format=RestaurantLocation,
    )
    

    location_dict[restaurant] = output.choices[0].message.parsed.planet

In [ ]:
location_dict

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/location_dict.json") as f:
    location_dict = json.load(f)  

### get plates ingredients

In [ ]:
class PlateIngredients(BaseModel):
    ingredients: list[str]

In [ ]:
ingredient_dict = {restaurant: dict() for restaurant in all_pdf_chunks.keys()}

for i, (restaurant, menu_chunks_dict) in enumerate(all_pdf_chunks.items(), start=1):
    print(f"{i} - {restaurant}")
    for dish, text in menu_chunks_dict.items():
        if dish == 'intro':
            continue
        
        output = client.beta.chat.completions.parse(
        model="gpt-4o",
        store=True,
        messages=[
            {
                "role": "system",
                "content": """
Sei un assistente che sta aiutando ad estrarre informazioni da alcuni menù di ristoranti scritti in italiano.
I ristoranti descritti ed i piatti contenuti nei menù sono inventati e si trovano in una ambientazione fantascientifica nello spazio, in diversi sistemi solari.
"""
            },
            {
                "role": "user",
                "content": f"""
Il seguente frammento di testo, preso dal menù del ristorante, descrive un piatto proposto dal ristorante.
All'interno del testo sono riportati gli ingredienti che compongo il piatto.
Restituiscimi una classe python contenente un attributo riportante la lista di stringhe relative agli ingredienti che compongono i piatti.
Considera che il ristorante ed il piatto fanno parte di un'ambientazione fantascientifica quindi gli ingredienti sono spesso inventati e non realistici.


### DESCRIZIONE DEL PIATTO:
{menu_chunks_dict[dish]}
"""
            }
        ],
        response_format=PlateIngredients,
    )
    

        ingredient_dict[restaurant][dish] = output.choices[0].message.parsed.ingredients

In [ ]:
ingredient_dict

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/ingredient_dict.json") as f:
    ingredient_dict = json.load(f)  

### get tecniques from Sirius Cosmo's manual

In [ ]:
file_path = '/Users/vanzettom/Documents/mattia/personale/hackapizza/Hackapizza Dataset/Misc/Manuale di Cucina.pdf'

doc = fitz.open(file_path)
text = "\n".join([page.get_text("text") for page in doc])
doc.close()

In [ ]:
row_list = [l for l in text.split("\n") if l != ""]
rows = (pd.Series(row_list))

In [ ]:
list_id_p_1 = list(rows[rows.apply(lambda x: x.lower().startswith('come funziona'))].index)

In [ ]:
sc_tecnique_list = [row_list[i] for i in [x-1 for x in list_id_p_1]]
len(tecnique_list)

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/sc_tecnique_list.json") as f:
    sc_tecnique_list = json.load(f)  

In [ ]:
", ".join(sc_tecnique_list) + "."

In [ ]:
sc_tecnique_dict = {
    "Tecniche di Preparazione": {
        "Tecniche di Marinatura":
            ['Marinatura a Infusione Gravitazionale',
            'Marinatura Temporale Sincronizzata',
            'Marinatura Psionica',
            "Marinatura tramite Reazioni d'Antimateria Diluite",
            'Marinatura Sotto Zero a Polarità Inversa'],
        "Tecniche di Affumicatura":
            ['Affumicatura a Stratificazione Quantica',
            'Affumicatura Temporale Risonante',
            'Affumicatura Psionica Sensoriale',
            'Affumicatura tramite Big Bang Microcosmico',
            'Affumicatura Polarizzata a Freddo Iperbarico',],
        "Tecniche di Fermentazione":
            ['Fermentazione Quantica a Strati Multiversali',
            'Fermentazione Temporale Sincronizzata',
            'Fermentazione Psionica Energetica',
            'Fermentazione tramite Singolarità',
            'Fermentazione Quantico Biometrica',],
        "Tecniche di Impasto": [
            'Impasto Gravitazionale Vorticoso',
            'Amalgamazione Sintetica Molecolare',
            'Impasto a Campi Magnetici Dualistici',
            'Sinergia Elettro-Osmotica Programmabile',
            'Modellatura Onirica Tetrazionale'],
        "tecniche di Surgelamento": [
            'Cryo-Tessitura Energetica Polarizzata',
            'Congelamento Bio-Luminiscente Sincronico',
            'Cristallizzazione Temporale Reversiva',
            'Congelazione Iperdimensionalmente Stratificata',
            'Surgelamento Antimaterico a Risonanza Inversa',],
        },
    "Tecniche di Cottura": {
        "Tecniche di Bollitura":
            ['Ebollizione Magneto-Cinetica Pulsante',
            'Bollitura Infrasonica Armonizzata',
            'Bollitura Termografica a Rotazione Veloce',
            'Bollitura Entropica Sincronizzata',
            'Idro-Cristallizzazione Sonora Quantistica',],
        "Tecniche di Grigliatura": [
            'Grigliatura a Energia Stellare DiV',
            'Grigliatura Plasma Sintetico Risonante',
            'Grigliatura Eletro-Molecolare a Spaziatura Variabile',
            'Grigliatura Tachionica Refrattaria',
            'Grigliatura Psionica Dinamica Ritmica',],
        "Tecniche di cottura al Forno":[
            'Cottura al Forno con Paradosso Temporale Cronospeculare',
            'Cottura al Forno con Paradosso Temporale Cronospeculare',
            'Cottura con Microonde Entropiche Sincronizzate',
            'Cottura a Forno Dinamico Inversionale',
            'Cottura Olografica Quantum Fluttuante',
            'Cottura Geomagnetica Psicosincronizzata',],
        "Tecniche di cottura al Vapore":[
            'Cottura a Vapore con Flusso di Particelle Isoarmoniche',
            'Cottura a Vapore Ecodinamico Bilanciato',
            'Cottura a Vapore Risonante Simbiotico',
            'Cottura a Vapore Termocinetica Multipla',
            'Cottura Idrodinamica Autoregolante',],
        "Tecniche di cottura Sottovuoto": [
            'Cottura Sottovuoto Antimateria',
            'Cottura Sottovuoto Multirealità Collassante',
            'Cottura Sottovuoto Frugale Energeticamente Negativa',
            'Cottura Sottovuoto Pulsar Magnetica',
            'Cottura Sottovuoto Bioma Sintetico',],
        "Tecniche di Saltare in Padella": [
            'Saltare in Padella Big Bang Termico',
            'Saltare in Padella Realtà Energetiche Parallele',
            'Saltare in Padella Singolarità Inversa',
            'Saltare in Padella Sinergia Psionica',
            'Saltare in Padella Classica',]
},
    "Tecniche Avanzate": {
        "Tecniche di Decostruzione": [
            'Decostruzione Atomica a Strati Energetici',
            'Decostruzione Magnetica Risonante',
            'Decostruzione Bio-Fotonica Emotiva',
            'Decostruzione Ancestrale',
            'Decostruzione Interdimensionale Lovecraftiana',],
        "Tecniche di Sferificazione": [
            'Sferificazione a Gravità Psionica Variabile',
            'Sferificazione Filamentare a Molecole Vibrazionali',
            'Sferificazione Cromatica Interdimensionale',
            'Sferificazione con Campi Magnetici Entropici',
            'Sferificazione tramite Matrici Biofotiche',],
        "Tecniche di Taglio": [
            'Taglio Dimensionale a Lame Fotofiliche',
            'Taglio a Risonanza Sonica Rigenerativa',
            'Affettamento a Pulsazioni Quantistiche',
            'Taglio Sinaptico Biomimetico',
            'Incisione Elettromagnetica Plasmica']
    }
}

### get plates tecniques

In [ ]:
class PlateTecniques(BaseModel):
    tecniques: list[str]

In [ ]:
tecnique_dict = {restaurant: dict() for restaurant in all_pdf_chunks.keys()}

for i, (restaurant, menu_chunks_dict) in enumerate(all_pdf_chunks.items(), start=1):
    print(f"{i} - {restaurant}")
    for dish, text in menu_chunks_dict.items():
        if dish == 'intro':
            continue
        
        output = client.beta.chat.completions.parse(
        model="gpt-4o",
        store=True,
        messages=[
            {
                "role": "system",
                "content": """
Sei un assistente che sta aiutando ad estrarre informazioni da alcuni menù di ristoranti scritti in italiano.
I ristoranti descritti ed i piatti contenuti nei menù sono inventati e si trovano in una ambientazione fantascientifica nello spazio, in diversi sistemi solari.
"""
            },
            {
                "role": "user",
                "content": f"""
Il seguente frammento di testo, preso dal menù del ristorante, descrive un piatto proposto dal ristorante.
All'interno del testo sono riportati le tecniche utilizzate per preparare il piatto.
Restituiscimi una classe python contenente un attributo riportante la lista di stringhe relative alle tecniche che è necessario usare per preparare i piatti.
Considera che il ristorante ed il piatto fanno parte di un'ambientazione fantascientifica quindi le tecniche sono spesso inventate e non realistice.

Questa è una lista **non esaustiva** che contiene esempi di tecniche che potrebbero essere utilizzate per preparare un piatto: 
{", ".join(tecnique_list) + "."}

### DESCRIZIONE DEL PIATTO:
{menu_chunks_dict[dish]}
"""
            }
        ],
        response_format=PlateTecniques,
    )
    

        tecnique_dict[restaurant][dish] = output.choices[0].message.parsed.tecniques

In [ ]:
tecnique_dict

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/tecnique_dict.json") as f:
    tecnique_dict = json.load(f)  

### Create list of tecniques from Sirius Cosmo's manual needed for every plate

In [ ]:
tecnique_sc_dict = {restaurant: dict() for restaurant in all_pdf_chunks.keys()}

max_distance = 5

for i, (restaurant, plate_tecniques_dict) in enumerate(tecnique_dict.items(), start=1):
    print(f"{i} - {restaurant}")
    for plate, tecnique_list in plate_tecniques_dict.items():
        tecnique_sc_dict[restaurant][plate] = list()
        for tecnique_used in tecnique_list:
            if any([Levenshtein.distance(tecnique_used.strip().lower(), sc_tecnique.lower()) <= max_distance for sc_tecnique in sc_tecnique_list]):
                tecnique_sc_dict[restaurant][plate].append(tecnique_used)

In [ ]:
sc_tecnique_dict

In [ ]:
plate_tecnique_sc_dict = {restaurant: dict() for restaurant in all_pdf_chunks.keys()}

max_distance = 7

for i, (restaurant, plate_tecniques_dict) in enumerate(tecnique_dict.items(), start=1):
    print(f"{i} - {restaurant}")
    
    for plate, tecnique_list in plate_tecniques_dict.items():
        plate_tecnique_sc_dict[restaurant][plate] = dict()
        # qui ho piatto e lista di tecniche che usa
        
        for tecnique_category, d_tecnique_category in sc_tecnique_dict.items():
            plate_tecnique_sc_dict[restaurant][plate][tecnique_category] = dict()
            
            for sub_tecnique_category, list_sub_tecniques in d_tecnique_category.items():
                plate_tecnique_sc_dict[restaurant][plate][tecnique_category][sub_tecnique_category] = [
                    sc_tecnique for sc_tecnique in list_sub_tecniques
                    if any([Levenshtein.distance(t.strip().lower(), sc_tecnique.lower()) <= max_distance for t in tecnique_list])
                ]

In [ ]:
plate_tecnique_sc_dict

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/plate_tecnique_sc_dict.json") as f:
    plate_tecnique_sc_dict = json.load(f)  

### Insert which lincenses are needed to do specific tecniques

I used Claude app to produce the following dictionary:

In [ ]:
licenses_needed_by_tecnique_dict = {
    "Tecniche di Preparazione": {
        "Tecniche di Marinatura": {
            'Marinatura a Infusione Gravitazionale': [
                {"Licenza Gravitazionale (G)": 2}
            ],
            'Marinatura Temporale Sincronizzata': [
                {"Licenza Temporale (t)": 1}
            ],
            'Marinatura Psionica': [
                {"Licenza Psionica (P)": 3}
            ],
            "Marinatura tramite Reazioni d'Antimateria Diluite": [
                {"Licenza Antimateria (e+)": 1}
            ],
            'Marinatura Sotto Zero a Polarità Inversa': [
                {"Licenza Magnetica (Mx)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ]
        },
        "Tecniche di Affumicatura": {
            'Affumicatura a Stratificazione Quantica': [
                {"Licenza Quantistica (Q)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Affumicatura Temporale Risonante': [
                {"Licenza Temporale (t)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Affumicatura Psionica Sensoriale': [
                {"Licenza Psionica (P)": 2},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Affumicatura tramite Big Bang Microcosmico': [
                {"Licenza Antimateria (e+)": 1},
                {"Licenza Quantistica (Q)": 10},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Affumicatura Polarizzata a Freddo Iperbarico': [
                {"Licenza Magnetica (Mx)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ]
        },
        "Tecniche di Fermentazione": {
            'Fermentazione Quantica a Strati Multiversali': [
                {"Licenza Quantistica (Q)": 5},
                {"Livello di Sviluppo Tecnologico (LTK)": 6}
            ],
            'Fermentazione Temporale Sincronizzata': [
                {"Licenza Temporale (t)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Fermentazione Psionica Energetica': [
                {"Licenza Psionica (P)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Fermentazione tramite Singolarità': [
                {"Licenza Gravitazionale (G)": 3},
                {"Licenza Antimateria (e+)": 1},
                {"Licenza Magnetica (Mx)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 4}
            ],
            'Fermentazione Quantico Biometrica': [
                {"Licenza Quantistica (Q)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ]
        },
        "Tecniche di Impasto": {
            'Impasto Gravitazionale Vorticoso': [
                {"Licenza Gravitazionale (G)": 1}
            ],
            'Amalgamazione Sintetica Molecolare': [
                {"Licenza Quantistica (Q)": 3}
            ],
            'Impasto a Campi Magnetici Dualistici': [
                {"Licenza Magnetica (Mx)": 1}
            ],
            'Sinergia Elettro-Osmotica Programmabile': [
                {"Licenza Antimateria (e+)": 1}
            ],
            'Modellatura Onirica Tetrazionale': [
                {"Licenza Psionica (P)": 4},
                {"Licenza Quantistica (Q)": 4},
                {"Livello di Sviluppo Tecnologico (LTK)": 4}
            ]
        },
        "tecniche di Surgelamento": {
            'Cryo-Tessitura Energetica Polarizzata': [
                {"Licenza Antimateria (e+)": 1}
            ],
            'Congelamento Bio-Luminiscente Sincronico': [
                {"Licenza Luce (c)": 2},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Cristallizzazione Temporale Reversiva': [
                {"Licenza Temporale (t)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Congelazione Iperdimensionalmente Stratificata': [
                {"Licenza Quantistica (Q)": 5},
                {"Livello di Sviluppo Tecnologico (LTK)": 4}
            ],
            'Surgelamento Antimaterico a Risonanza Inversa': [
                {"Licenza Antimateria (e+)": 1},
                {"Licenza Magnetica (Mx)": 1},
                {"Licenza Quantistica (Q)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ]
        }
    },
    "Tecniche di Cottura": {
        "Tecniche di Bollitura": {
            'Ebollizione Magneto-Cinetica Pulsante': [
                {"Licenza Magnetica (Mx)": 1}
            ],
            'Bollitura Infrasonica Armonizzata': [
                {"Licenza Quantistica (Q)": 3}
            ],
            'Bollitura Termografica a Rotazione Veloce': [
                {"Licenza Luce (c)": 1}
            ],
            'Bollitura Entropica Sincronizzata': [
                {"Licenza Quantistica (Q)": 4},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Idro-Cristallizzazione Sonora Quantistica': [
                {"Licenza Quantistica (Q)": 5},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ]
        },
        "Tecniche di Grigliatura": {
            'Grigliatura a Energia Stellare DiV': [
                {"Licenza Luce (c)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Grigliatura Plasma Sintetico Risonante': [
                {"Licenza Antimateria (e+)": 1},
                {"Licenza Magnetica (Mx)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Grigliatura Eletro-Molecolare a Spaziatura Variabile': [
                {"Licenza Quantistica (Q)": 3}
            ],
            'Grigliatura Tachionica Refrattaria': [
                {"Licenza Temporale (t)": 2},
                {"Licenza Luce (c)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 5}
            ],
            'Grigliatura Psionica Dinamica Ritmica': [
                {"Licenza Psionica (P)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ]
        },
        "Tecniche di cottura al Forno": {
            'Cottura al Forno con Paradosso Temporale Cronospeculare': [
                {"Licenza Temporale (t)": 3},
                {"Licenza Quantistica (Q)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Cottura con Microonde Entropiche Sincronizzate': [
                {"Licenza Quantistica (Q)": 4}
            ],
            'Cottura a Forno Dinamico Inversionale': [
                {"Licenza Antimateria (e+)": 1}
            ],
            'Cottura Olografica Quantum Fluttuante': [
                {"Licenza Quantistica (Q)": 5},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Cottura Geomagnetica Psicosincronizzata': [
                {"Licenza Psionica (P)": 5},
                {"Licenza Magnetica (Mx)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 4}
            ]
        },
        "Tecniche di cottura al Vapore": {
            'Cottura a Vapore con Flusso di Particelle Isoarmoniche': [],
            'Cottura a Vapore Ecodinamico Bilanciato': [],
            'Cottura a Vapore Risonante Simbiotico': [
                {"Licenza Psionica (P)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Cottura a Vapore Termocinetica Multipla': [],
            'Cottura Idrodinamica Autoregolante': [
                {"Licenza Antimateria (e+)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ]
        },
        "Tecniche di cottura Sottovuoto": {
            'Cottura Sottovuoto Antimateria': [
                {"Licenza Gravitazionale (G)": 1},
                {"Licenza Antimateria (e+)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Cottura Sottovuoto Multirealità Collassante': [
                {"Licenza Gravitazionale (G)": 1},
                {"Licenza Quantistica (Q)": 5},
                {"Livello di Sviluppo Tecnologico (LTK)": 6}
            ],
            'Cottura Sottovuoto Frugale Energeticamente Negativa': [
                {"Licenza Gravitazionale (G)": 1},
                {"Licenza Quantistica (Q)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Cottura Sottovuoto Pulsar Magnetica': [
                {"Licenza Gravitazionale (G)": 1},
                {"Licenza Magnetica (Mx)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Cottura Sottovuoto Bioma Sintetico': [
                {"Licenza Psionica (P)": 2},
                {"Licenza Gravitazionale (G)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ]
        },
        "Tecniche di Saltare in Padella": {
            'Saltare in Padella Big Bang Termico': [
                {"Licenza Gravitazionale (G)": 3},
                {"Licenza Antimateria (e+)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 5}
            ],
            'Saltare in Padella Realtà Energetiche Parallele': [
                {"Licenza Quantistica (Q)": 6},
                {"Livello di Sviluppo Tecnologico (LTK)": 4}
            ],
            'Saltare in Padella Singolarità Inversa': [
                {"Licenza Gravitazionale (G)": 3},
                {"Licenza Antimateria (e+)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 4}
            ],
            'Saltare in Padella Sinergia Psionica': [
                {"Licenza Psionica (P)": 5},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Saltare in Padella Classica': []
        }
    },
    "Tecniche Avanzate": {
        "Tecniche di Decostruzione": {
            'Decostruzione Atomica a Strati Energetici': [
                {"Licenza Antimateria (e+)": 1},
                {"Licenza Quantistica (Q)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Decostruzione Magnetica Risonante': [
                {"Licenza Magnetica (Mx)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Decostruzione Bio-Fotonica Emotiva': [
                {"Licenza Psionica (P)": 3},
                {"Licenza Luce (c)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Decostruzione Ancestrale': [
                {"Licenza Temporale (t)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Decostruzione Interdimensionale Lovecraftiana': [
                {"Licenza Quantistica (Q)": 7},
                {"Livello di Sviluppo Tecnologico (LTK)": 6}
            ]
        },
        "Tecniche di Sferificazione": {
            'Sferificazione a Gravità Psionica Variabile': [
                {"Licenza Psionica (P)": 4},
                {"Licenza Gravitazionale (G)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Sferificazione Filamentare a Molecole Vibrazionali': [
                {"Licenza Magnetica (Mx)": 1},
                {"Licenza Quantistica (Q)": 4},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Sferificazione Cromatica Interdimensionale': [
                {"Licenza Quantistica (Q)": 6},
                {"Licenza Luce (c)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Sferificazione con Campi Magnetici Entropici': [
                {"Licenza Magnetica (Mx)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ],
            'Sferificazione tramite Matrici Biofotiche': [
                {"Licenza Magnetica (Mx)": 1},
                {"Licenza Quantistica (Q)": 3},
                {"Licenza Luce (c)": 2},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ]
        },
        "Tecniche di Taglio": {
            'Taglio Dimensionale a Lame Fotofiliche': [
                {"Licenza Quantistica (Q)": 6},
                {"Licenza Luce (c)": 3},
                {"Livello di Sviluppo Tecnologico (LTK)": 4}
            ],
            'Taglio a Risonanza Sonica Rigenerativa': [],
            'Affettamento a Pulsazioni Quantistiche': [
                {"Licenza Quantistica (Q)": 4},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Taglio Sinaptico Biomimetico': [
                {"Licenza Psionica (P)": 4},
                {"Livello di Sviluppo Tecnologico (LTK)": 3}
            ],
            'Incisione Elettromagnetica Plasmica': [
                {"Licenza Antimateria (e+)": 1},
                {"Licenza Magnetica (Mx)": 1},
                {"Livello di Sviluppo Tecnologico (LTK)": 2}
            ]
        }
    }
}

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/licenses_needed_by_tecnique_dict.json") as f:
    licenses_needed_by_tecnique_dict = json.load(f)  

In [ ]:
licenses_needed_by_tecnique_dict

### Insert geo info

In [ ]:
distances = pd.read_csv('/Users/vanzettom/Documents/mattia/personale/hackapizza/Hackapizza Dataset/Misc/Distanze.csv')
distances.shape

In [ ]:
location_dict

In [ ]:
distances_dict = {restaurant: dict() for restaurant in all_pdf_chunks.keys()}

for restaurant in distances_dict:
    distances_dict[restaurant] = dict(zip(
        [c for c in distances.columns if c != "/"],
        [float(n) for n in distances.loc[distances['/'] == location_dict.get(restaurant), [c for c in distances.columns if c != "/"]].values[0]]
    ))

In [ ]:
distances_dict

### Put all togheter in one single dictionary

In [ ]:
map_correct_name = dict()

regex = r"[🌱🌈🪐]"

result = re.sub(regex, "", text).strip()

for restaurant, map_dict in mapping_name_found.items():
    map_dict = {re.sub(regex, "", k).strip(): re.sub(regex, "", v).strip() for k, v in map_dict.items()}
    map_correct_name.update(dict(zip(map_dict.values(), map_dict.keys())))

In [ ]:
plate_info_dictionary = dict()

def estrai_licenze(dizionario_base, dizionario_completo):
    licenze_risultato = []
    
    for categoria, sotto_categorie in dizionario_base.items():
        if categoria in dizionario_completo:
            for sotto_categoria, tecniche in sotto_categorie.items():
                if sotto_categoria in dizionario_completo[categoria]:
                    for tecnica in tecniche:
                        if tecnica in dizionario_completo[categoria][sotto_categoria]:
                            licenze = dizionario_completo[categoria][sotto_categoria][tecnica]
                            for lic in licenze:
                                licenze_risultato.append(lic)
    
    return licenze_risultato

def dict_in_list(target_dict, dict_list, tolerance=2):
    for d in dict_list:
        for key_t, value_t in target_dict.items():
            for key_d, value_d in d.items():
                if Levenshtein.distance(key_t, key_d) < tolerance and value_t <= value_d:
                    return True
    return False

def all_dicts_in_list(small_list, big_list, tolerance=2):
    return all(dict_in_list(d, big_list, tolerance) for d in small_list)

for restaurant, plate_tecnique_dict in tecnique_dict.items():
    chef_non_in_regola = list()

    for plate, tecnique_list in plate_tecnique_dict.items():
        
        plate_info_dictionary[plate] = dict()
        plate_info_dictionary[plate]["dati_ristorante"] = dict()
        plate_info_dictionary[plate]["dati_ristorante"]["nome_ristorante"] = restaurant
        plate_info_dictionary[plate]["dati_ristorante"]["chef"] = dict()
        plate_info_dictionary[plate]["dati_ristorante"]["chef"]["nome"] = chef_data_dict_clean[restaurant]["chef_name"]
        plate_info_dictionary[plate]["dati_ristorante"]["chef"]["lista_licenze"] = eval(chef_data_dict_clean[restaurant]["license_list"])
        plate_info_dictionary[plate]["dati_ristorante"]["chef"]["lista_licenze"] = [
            {list(d.keys())[0]: float(list(d.values())[0])} for d in plate_info_dictionary[plate]["dati_ristorante"]["chef"]["lista_licenze"]
        ]
        plate_info_dictionary[plate]["dati_ristorante"]["pianeta"] = location_dict[restaurant]
        plate_info_dictionary[plate]["dati_ristorante"]["distanze_da_altri_pianeti"] = distances_dict[restaurant]
        
        plate_info_dictionary[plate]["tecniche_utilizzate"] = tecnique_list
        plate_info_dictionary[plate]["tecniche_utilizzate_del_Sirius_Cosmo"] = plate_tecnique_sc_dict[restaurant][plate]
        
        license_needed = estrai_licenze(plate_info_dictionary[plate]["tecniche_utilizzate_del_Sirius_Cosmo"], licenses_needed_by_tecnique_dict)
        
        plate_info_dictionary[plate]["licenze_necessarie_per_la_preparazione"] = license_needed
        
        if all_dicts_in_list(license_needed, plate_info_dictionary[plate]["dati_ristorante"]["chef"]["lista_licenze"], tolerance=2):
            plate_info_dictionary[plate]["dati_ristorante"]["chef"]["licenze_ok"] = True
        else:
            plate_info_dictionary[plate]["dati_ristorante"]["chef"]["licenze_ok"] = False
          
        plate_info_dictionary[plate]["ingredienti"] = [
            ing for ing in ingredient_dict[restaurant][plate]
            if not any([err in ing for err in ['decostruzione', 'sferificazione','grigliatura','marinatura', 'fermentazione', 'amalgamazione', 'affumicatura', 'cottura', 'congelamento', 'tessitura']])
        ]
        
        text_chunk = all_pdf_chunks[restaurant][plate]#[mapping_name_found[restaurant][plate]]
        
            
        if "Questo piatto può essere mangiato dai membri dell'Ordine della Galassia di Andromeda.".lower() in text_chunk:
            plate_info_dictionary[plate]['mangiabile_da_ordine_galassia_andromeda'] = True
        else:
            plate_info_dictionary[plate]['mangiabile_da_ordine_galassia_andromeda'] = False
        
        if "Questo piatto può essere mangiato dai membri dell'Ordine dei Naturalisti.".lower() in text_chunk:
            plate_info_dictionary[plate]['mangiabile_da_ordine_naturisti'] = True
        else:
            plate_info_dictionary[plate]['mangiabile_da_ordine_naturisti'] = False
        
        if "questo piatto può essere mangiato dai membri dell'ordine degli armonisti".lower() in text_chunk:
            plate_info_dictionary[plate]['mangiabile_da_ordine_armonisti'] = True

        else:
            plate_info_dictionary[plate]['mangiabile_da_ordine_armonisti'] = False

        
plate_info_dictionary = {map_correct_name.get(plate): d for plate, d in plate_info_dictionary.items()}

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/plate_info_dictionary.json", "w", encoding="utf-8") as f:
    json.dump(plate_info_dictionary, f, ensure_ascii=False, indent=4)

### Preparation of text chunks that could be useful to understand if a plate falls within the user's request

In [ ]:
map_correct_name['quadrifonia cosmica: sinfonia di sapori e dimensioni']

In [ ]:
plate_chunks_rag = dict()

for restaurant, dict_paragraphs in all_pdf_chunks.items():
    for plate, chunk in dict_paragraphs.items():
        if plate != 'intro':
            plate_chunks_rag[map_correct_name[plate]] = f"""
Il piatto è offerto dal Ristorante {restaurant}.

Il ristorante si trova sul pianeta: {location_dict[restaurant]}.

Descrizione piatto:\n{chunk}

Descrizione dello chef:\n{dict_paragraphs['intro']}
"""

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/plate_chunks_rag.json", "w", encoding="utf-8") as f:
    json.dump(plate_chunks_rag, f, ensure_ascii=False, indent=4)

In [ ]:
file_path = '/Users/vanzettom/Documents/mattia/personale/hackapizza/Hackapizza Dataset/Codice Galattico/Codice Galattico.docx'

doc = fitz.open(file_path)
codice_galattico = "\n".join([page.get_text("text") for page in doc])
doc.close()

In [ ]:
print(codice_galattico)

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/codice_galattico.json", "w", encoding="utf-8") as f:
    json.dump(codice_galattico, f, ensure_ascii=False, indent=4)

In [ ]:
file_path = '/Users/vanzettom/Documents/mattia/personale/hackapizza/Hackapizza Dataset/Misc/Manuale di Cucina.pdf'

doc = fitz.open(file_path)
manuale_sirius_cosmo = "\n".join([page.get_text("text") for page in doc])
doc.close()

In [ ]:
print(manuale_sirius_cosmo)

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/manuale_sirius_cosmo.json", "w", encoding="utf-8") as f:
    json.dump(manuale_sirius_cosmo, f, ensure_ascii=False, indent=4)

In [ ]:
distanze = """

### DISTANZE TRA PIANETI:

|            | Tatooine | Asgard | Namecc | Arrakis | Krypton | Pandora | Cybertron | Ego | Montressosr | Klyntar |
|------------|---------|--------|--------|---------|---------|---------|-----------|-----|-------------|---------|
| **Tatooine**    | 0       | 695    | 641    | 109     | 661     | 1130    | 344       | 835 | 731         | 530     |
| **Asgard**      | 695     | 0      | 550    | 781     | 188     | 473     | 493       | 156 | 240         | 479     |
| **Namecc**      | 641     | 550    | 0      | 651     | 367     | 987     | 728       | 688 | 767         | 845     |
| **Arrakis**     | 109     | 781    | 651    | 0       | 727     | 1227    | 454       | 926 | 834         | 640     |
| **Krypton**     | 661     | 188    | 367    | 727     | 0       | 626     | 557       | 321 | 422         | 599     |
| **Pandora**     | 1130    | 473    | 987    | 1227    | 626     | 0       | 847       | 317 | 413         | 731     |
| **Cybertron**   | 344     | 493    | 728    | 454     | 557     | 847     | 0         | 594 | 434         | 186     |
| **Ego**         | 835     | 156    | 688    | 926     | 321     | 317     | 594       | 0   | 215         | 532     |
| **Montressosr** | 731     | 240    | 767    | 834     | 422     | 413     | 434       | 215 | 0           | 331     |
| **Klyntar**     | 530     | 479    | 845    | 640     | 599     | 731     | 186       | 532 | 331         | 0       |
"""

In [ ]:
with open("/Users/vanzettom/Documents/mattia/personale/hackapizza/data_preprocessed/distanze.json", "w", encoding="utf-8") as f:
    json.dump(distanze, f, ensure_ascii=False, indent=4)